# 3. Machine Learning For classification

- Dataset: https://www.kaggle.com/blastchar/telco-customer-churn
- https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv

## 3.1 Churn prediction project

In [73]:
# Importation of packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
data = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv"

In [75]:
!wget $data -O data.csv

--2025-10-14 12:05:04--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: 'data.csv'

     0K .......... .......... .......... .......... ..........  5%  367K 2s
    50K .......... .......... .......... .......... .......... 10%  509K 2s
   100K .......... .......... .......... .......... .......... 15%  936K 2s
   150K .......... .......... .......... .......... .......... 20%  426K 2s
   200K .......... .......... .......... .......... .......... 26% 2.00M 1s
   250K .......... .......... .......... .......... .......... 31% 1.00M 1s
   300K .......... .......... ......

In [98]:
df = pd.read_csv('data.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Preparation

In [99]:
df.columns = df.columns.str.lower().str.replace(" ","_")

In [100]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [101]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [102]:
df.totalcharges = df.totalcharges.fillna(0)

In [103]:
df.churn.head()

0     No
1     No
2    Yes
3     No
4    Yes
Name: churn, dtype: object

In [105]:
df.churn = df.churn.map({'No': 0, 'Yes': 1})
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int64

### Setting Validation Framework
- Splitting Data into Train/Split/Validation Using Scikit_Learn

In [106]:
from sklearn.model_selection import train_test_split

In [107]:
# Splitting the Data Into Train and test data 
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)


In [108]:
# Splitting the full train data into Train and val
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [109]:
len(df_test), len(df_train), len(df_val)

(1409, 4225, 1409)

In [110]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [111]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [112]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

## EDA
- Missing Values check
- look at the target variable(churn)
- look at numerical and categorical variables

In [113]:
df_full_train = df_full_train.reset_index(drop=True)

In [114]:
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [115]:
df_full_train.churn.value_counts(normalize=True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [116]:
df_full_train.churn.mean()

np.float64(0.26996805111821087)

In [117]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [119]:
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [120]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [132]:
for col in categorical:
    # print(col)  # print column name
    print(df_full_train[col].value_counts())
    print()

gender
Male      2838
Female    2796
Name: count, dtype: int64

seniorcitizen
0    4722
1     912
Name: count, dtype: int64

partner
No     2932
Yes    2702
Name: count, dtype: int64

dependents
No     3968
Yes    1666
Name: count, dtype: int64

phoneservice
Yes    5087
No      547
Name: count, dtype: int64

multiplelines
No                  2700
Yes                 2387
No phone service     547
Name: count, dtype: int64

internetservice
Fiber optic    2479
DSL            1934
No             1221
Name: count, dtype: int64

onlinesecurity
No                     2801
Yes                    1612
No internet service    1221
Name: count, dtype: int64

onlinebackup
No                     2498
Yes                    1915
No internet service    1221
Name: count, dtype: int64

deviceprotection
No                     2473
Yes                    1940
No internet service    1221
Name: count, dtype: int64

techsupport
No                     2781
Yes                    1632
No internet service    12

## Feature Importance: Churn rate and Risk Ratio
Feature importance analysis (part of EDA) - identifying which features affect our target variable

- Churn rate
- Risk ratio
- Mutual information - later

*Churn Rate*

In [121]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-PPTJY,Male,0,Yes,Yes,12,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.70,258.35,0
1,6261-RCVNS,Female,0,No,No,42,Yes,No,DSL,Yes,...,Yes,Yes,No,Yes,One year,No,Credit card (automatic),73.90,3160.55,1
2,2176-OSJUV,Male,0,Yes,No,71,Yes,Yes,DSL,Yes,...,No,Yes,No,No,Two year,No,Bank transfer (automatic),65.15,4681.75,0
3,6161-ERDGD,Male,0,Yes,Yes,71,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,No,Electronic check,85.45,6300.85,0
4,2364-UFROM,Male,0,No,No,30,Yes,No,DSL,Yes,...,No,Yes,Yes,No,One year,No,Electronic check,70.40,2044.75,0


In [135]:
global_churn =  df_full_train['churn'].mean()
global_churn

np.float64(0.26996805111821087)

In [137]:
churn_female = df_full_train[df_full_train.gender == "Female"].churn.mean()
churn_female

np.float64(0.27682403433476394)

In [138]:
churn_male = df_full_train[df_full_train.gender == "Male"].churn.mean()
churn_male

np.float64(0.2632135306553911)

In [142]:
print(global_churn - churn_female)
print(global_churn - churn_male)

-0.006855983216553063
0.006754520462819769


In [143]:
df_full_train.partner.value_counts()

partner
No     2932
Yes    2702
Name: count, dtype: int64

In [146]:
churn_dependents = df_full_train[df_full_train.dependents == "Yes"].churn.mean()
churn_dependents

np.float64(0.16566626650660263)

In [148]:
churn_non_dependents = df_full_train[df_full_train.dependents == "No"].churn.mean()
churn_non_dependents

np.float64(0.3137600806451613)

In [150]:
print(global_churn - churn_non_dependents)
print(global_churn - churn_dependents)


-0.04379202952695044
0.10430178461160824


Risk Ratio

In [152]:
print(churn_dependents/global_churn)
print(churn_non_dependents/global_churn)

0.6136513777108475
1.162211896354266


SELECT
    gender,
    AVG(churn),
    AVG(churn) - global_churn AS diff,
    AVG(churn) / global_churn AS risk
FROM
    data
GROUP BY
    gender;

In [158]:
from IPython.display import display

In [161]:
for col in categorical:
    print(col)
    df_group = df_full_train.groupby(col).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn 
    df_group['risk'] = df_group['mean'] / global_churn 
    display(df_group)
    print()
    print() 

gender


,mean,count,diff,risk
gender,,,,
Female,0.276824,2796,0.006856,1.025396
Male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
Month-to-month,0.431701,3104,0.161733,1.599082
One year,0.120573,1186,-0.149395,0.446621
Two year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
No,0.172071,2313,-0.097897,0.637375
Yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
Bank transfer (automatic),0.168171,1219,-0.101797,0.622928
Credit card (automatic),0.164339,1217,-0.105630,0.608733
Electronic check,0.455890,1893,0.185922,1.688682
Mailed check,0.193870,1305,-0.076098,0.718121


## Feature Importance: Mutual Information
Mutual information - a concept from information theory, it tells us how mcuh we can learn about one variable if we know the value of another


In [162]:
from sklearn.metrics import mutual_info_score

In [165]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.0983203874041556

In [167]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [168]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## Feature importance: Correlation
how about the numerical columns?
- Correlation coefficient - https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

In [169]:
df_full_train.tenure.max()

np.int64(72)

In [172]:
df_full_train[numerical].corrwith(df_full_train.churn).abs()

tenure            0.351885
monthlycharges    0.196805
totalcharges      0.196353
dtype: float64

In [173]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

np.float64(0.5953420669577875)

In [174]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean()

np.float64(0.3994413407821229)

In [175]:
df_full_train[df_full_train.tenure > 12].churn.mean()

np.float64(0.17634908339788277)

## One-hot Encoding
using Scikit-Learn to encode categorical features

In [176]:
from sklearn.feature_extraction import DictVectorizer

In [183]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
x_val = dv.transform(val_dict)

## 3.9 Logistic Regression
- Binary classification
- Linear Vs Logistics Regression